In [1]:
from pymatgen.io.openmm.generators import OpenMMSolutionGen
import numpy as np

%load_ext autoreload
%autoreload 2

### Generators

A pymatgen InputSet contains all of the files needed to set up and run a simulation. Writing those files by hand would be incredibly tedious, so we use a Generator to create the InputSet. The Generator is instantiated with simulation properties as input, like temperature and force field. Once it is created, it can generated many InputSets with different sets of molecules.

In [3]:
generator = OpenMMSolutionGen(
    default_force_field="sage",
    temperature= 298,
    step_size= 0.001,
    default_charge_method= "am1bcc",
)

All of the keywords above are just the defaults, we could also simply call:

In [4]:
generator = OpenMMSolutionGen()

### Input Set

An InputSet is created by calling `generator.get_input_set()` and providing dicts defining each molecule and the density of the solution.Below we show a bare minimum example of what that can look like.

In [5]:
input_mol_dicts = [
    {"smile": "O", "count": 200},
    {"smile": "CCO", "count": 20},
]

input_set = generator.get_input_set(input_mol_dicts, density=1)

As you can see, the input_mol_dicts is a list of dictionaries, each dictionary must include a `smile` and `count` but there a number of optional keywords. Let's use the `name`, `charge_scaling`, `geometries`, and `partial_charges` keywords to create a new, more customized solution.

In [6]:
from pymatgen.io.openmm.tests.datafiles import PF6_charges, PF6_xyz

pf6_charge_array = np.load(PF6_charges)

input_mol_dicts = [
    {
        "smile": "C1COC(=O)O1",
        "count": 100,
        "name": "EC",  # this name is used for all downstream references
    },
    {
        "smile": "[Li+]",
        "count": 10,
        "charge_scaling": 0.8,  # this will scale the charges to 0.8 of their normal value
    },
    {
        "smile": "F[P-](F)(F)(F)(F)F",
        "count": 10,
        "charge_scaling": 0.8,
        "geometries": [PF6_xyz],  # we can supply our own initial geometries
        "partial_charges": pf6_charge_array,  # the order of charges MUST MATCH the order of charges in the geometries
    },
]

For a complete definition of the input_mol_dict specification and options, see `pymatgen.io.openmm.schema.InputMoleculeSpec`.

In [7]:
input_set = generator.get_input_set(input_mol_dicts, density=1)

[08:48:41] *****: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.
